In [121]:
import pandas as pd
import pandas as pd

df_conf = pd.read_csv('data/df_conf.csv', index_col=0)
df_manual = pd.read_csv('data_processed/data_manual.csv', index_col=0)
df_manual['source'] = 'manual'
df_manual.rename({'label': 'ai_generated'}, axis=1, inplace=True)
df_manual['ai_generated'] = df_manual['ai_generated'].astype(int)
df_manual['ours'] =0
df_manual['val'] = 0
df_manual['most confident'] = '0'
df_manual['confidence'] = 0
df_manual.reset_index(drop=False, inplace=True)
df_manual['text'] = df_manual['text'].astype(str)

# remove leading '\n' from text col (only at the beginning)
df_manual['text'] = df_manual['text'].apply(lambda x: x[1:] if x[0]=='\n' else x)
df_manual


# # concat
df = pd.concat([df_conf, df_manual], axis=0)
df

# save 
df.to_csv('data/data_concat.csv')
df

,text,ai_generated,source,ours,val,most confident,confidence
0,Dansk Varmblods kåringskommission fældede lørd...,0,data/heste-nettet-nyheder/,3.157006,-3.135451,ours,-6.292458
1,Hotellet tilbyder 145 moderne værelser og suit...,1,data/from_big_data_file_ai/gpt-3.5-turbo-0613/,2.563214,-2.441174,ours,-5.004388
2,Hos Hobby Haven har vi engagerede medarbejdere...,0,data/from_big_data_file/,0.013200,0.365958,val,0.352758
3,"Kære landets borgere,\n\nJeg står i dag foran ...",1,data/from_big_data_file_ai/gpt-3.5-turbo-0613/,0.847833,-0.846252,ours,-1.694084
4,Play'n GO er en førende virksomhed inden for u...,1,data/from_big_data_file_ai/gpt-3.5-turbo/,1.450386,-1.252852,ours,-2.703237
...,...,...,...,...,...,...,...
1006,Godt du reagerer – i din alder skal testiklern...,0,manual,0.000000,0.000000,0,0.000000
1007,"Med to tidskørsler, nogle korte etaper samt fl...",0,manual,0.000000,0.000000,0,0.000000
1008,Interviewer: Dette rejser et interessant spørg...,1,manual,0.000000,0.000000,0,0.000000
1009,Har du optjent en aktieindkomst på under 58.90...,0,manual,0.000000,0.000000,0,0.000000


In [122]:
df_small = pd.read_csv('data.csv')
df_small

,text,is_generated
0,"Det gør firmaerne, der står bag AI'en - eksemp...",0
1,"Tror det bliver for kedeligt, hvis vi bare læs...",0
2,"Godt spørgsmål! Det er nemlig meget, meget svæ...",0
3,"Ja, det er begyndt at ske. Det er dog ikke så ...",1
4,Det vil jeg tro - uden at vide det helt præcis...,0
5,"Selvom det er svært at forudsige, hvornår AI v...",1
6,"Det afhænger nok meget af, hvor fremme i skoen...",0
7,"En tech-korrespondent er en journalist, der dæ...",1
8,"ChatGPT er en generativ AI-model, der lærer at...",1
9,"Hej, den har læst en masse tekst og så regner ...",0


* For this to work, we need to process big batches (100 lines)
* So fine tuning must also use this format
* or can we make parallel api calls?

In [123]:
from openai import OpenAI
import os
import dotenv
import numpy as np
import concurrent.futures

import time

#load .env file
dotenv.load_dotenv()
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

def predict_batches(texts, model, batch_size=40):
    '''
    Take a list of strings, predict the labels and return a list of predictions 
    '''
    predictions = {}
    for i in range(0, len(texts), batch_size):
        print(i)
        print(i)
        print(i)
        batch = texts[i:i+batch_size]
        
        combined_string = '\n'.join([str(j+1) + '. ' + '$$$' +text + '$$$' for j, text in enumerate(batch)])
    
        completion = client.chat.completions.create(
        model=model,
        messages=[
            {
                "role": "system", 
                "content": """Du får udleveret mange tekststykke. Du skal vurdere, om hver tekst er skrevet af et menneske eller en AI. Du skal svare 0, hvis du tror, ​​det er skrevet af et menneske, og 1, hvis du tror, ​​det er skrevet af en AI. Kun ét ciffer per text er tilladt. Du vil få input som er afmærket med et tal og 3 dollartegn ($$$) i starten og slutningen. Der skal til mellem hvert svar være et linjeskift (\n).
                
                Eksempel:
                1. $$$Tekst 1 er en lang text ....
                den dækker flere linjer
                Og har alverdens tegn i sig
                $$$
                2. $$$Tekst 2$$$

                3. $$$Tekst 3$$$
                ....
                Og du skal svare med:
                1. 0
                2. 1
                3. 0
                ...."""},
            {"role": "user", "content": combined_string}
        ]
        )
        print(completion.choices[0].message.content.split('\n'))
        

def predict_batch(text_list, model='ft:gpt-3.5-turbo-1106:personal::8LbDMp07'
                  #"ft:gpt-3.5-turbo-0613:personal::8KV9rODr"
                  ):
    try:
        # prompts
        system_prompt = """Du får udleveret mange tekststykke. Du skal vurdere, om hver tekst er skrevet af et menneske eller en AI. Du skal svare 0, hvis du tror, ​​det er skrevet af et menneske, og 1, hvis du tror, ​​det er skrevet af en AI. Kun ét ciffer per text er tilladt. Du vil få input som er afmærket med et tal og 3 dollartegn ($$$) i starten og slutningen. Der skal til mellem hvert svar være et linjeskift (\n).
                
                Eksempel:
                1. $$$Tekst 1 er en lang text ....
                den dækker flere linjer
                Og har alverdens tegn i sig
                $$$
                2. $$$Tekst 2$$$

                3. $$$Tekst 3$$$
                ...

                Og du skal svare med:
                1. 0
                2. 1
                3. 0
                ...
                """

        text_list_formatted = [str(i+1) + '. $$$' + text + '$$$' for i, text in enumerate(text_list)]
        text = '\n\n'.join(text_list_formatted)
        # print(text)

        completion = client.chat.completions.create(
            model=model,
            messages=[
                {"role": "system", "content": system_prompt},
                {"role": "user", "content": text}
            ]
        )
        content = completion.choices[0].message.content
        # print('##########################################')
        # print(content)
        # print('##########################################')
        content_lines = content.split('\n')
        predictions = {}
        for i, line in enumerate(content_lines):
            try:
                num = line.split('. ')[0]
            except:
                num = i
                print('Could not split line:', line)

            try:
                pred = line.split('. ')[1]

            except:
                pred = -1
                print('Could not split line:', line)

            predictions[int(num)-1] = pred
                
        # print(predictions)
            
        
        pred_clean = []
        for i in range(len(text_list)):
            if i not in predictions.keys():
                pred_clean.append(-1)
                print('Missing prediction for text number', i, 'with text:', text_list[i])
            else:
                try:
                    pred_clean.append(int(predictions[i]))
                except:
                    pred_clean.append(-1)
                    print('Cleaning failed for text number', i, 'with text:', text_list[i])
        # convert to int
        pred_clean = [int(x) for x in pred_clean]
        return pred_clean

    except Exception as e:
        print("Rate limit reached, sleeping for 30 milliseconds")
        time.sleep(0.03)
        return predict_batch(text_list, model)  # Retry the request

def predict_parallel(text_batches):
    with concurrent.futures.ThreadPoolExecutor() as executor:
        results = list(executor.map(predict_batch, text_batches))
    return results

def clean_up(x):
    if x == '0' or x == 0:
        return 0

    if x == '1' or x == 1:
        return 1

    return 1


# use predict batch on df_small
df_small['pred'] = predict_batch(df_small['text'].tolist()[:])
df_small

,text,is_generated,pred
0,"Det gør firmaerne, der står bag AI'en - eksemp...",0,0
1,"Tror det bliver for kedeligt, hvis vi bare læs...",0,0
2,"Godt spørgsmål! Det er nemlig meget, meget svæ...",0,1
3,"Ja, det er begyndt at ske. Det er dog ikke så ...",1,1
4,Det vil jeg tro - uden at vide det helt præcis...,0,1
5,"Selvom det er svært at forudsige, hvornår AI v...",1,1
6,"Det afhænger nok meget af, hvor fremme i skoen...",0,1
7,"En tech-korrespondent er en journalist, der dæ...",1,1
8,"ChatGPT er en generativ AI-model, der lærer at...",1,1
9,"Hej, den har læst en masse tekst og så regner ...",0,0


In [126]:
# load 
import numpy as np
# load texts and labels
df = pd.read_csv('data/data_concat.csv', index_col=0)
df[df['confidence'] >= 0]
# text = df_conf[df_conf['confidence'] > 0]['text'].values
# labels = df_conf[df_conf['confidence'] > 0]['ai_generated'].values

data = df[df['confidence'] >= 0][['text', 'ai_generated']].copy()
data.rename(columns={'ai_generated': 'label'}, inplace=True)

# to fix: TypeError: expected string or bytes-like object
data['text'] = data['text'].astype(str)

# text, labels = load_data()
# data = pd.DataFrame({'text': text, 'label': labels})
# random shuffle
data = data.sample(frac=1).reset_index(drop=True)


In [127]:
data2 = pd.read_csv('data/validation_labeled.csv', index_col=0)
data2.rename(columns={'labels': 'label'}, inplace=True)



data_concat = pd.concat([data, data2], axis=0)
print(len(data_concat))

3069


In [128]:
# save first 80% as final_training_data.csv
data_concat.iloc[:int(len(data_concat)*0.93)].to_csv('data/final_training_data.csv')

# save last 20% as final_validation_data.csv
data_concat.iloc[int(len(data_concat)*0.93):].to_csv('data/final_validation_data.csv')

In [129]:
data = pd.read_csv('data/final_training_data.csv', index_col=0)
data['label'] = data['label'].astype(str)
data['text'] = data['text'].astype(str)

data_val = pd.read_csv('data/final_validation_data.csv', index_col=0)
data_val['label'] = data_val['label'].astype(str)
data_val['text'] = data_val['text'].astype(str)

## Fine-tuning GPT

In [108]:
import json
def save_to_json(data, n=20, out_file='data.jsonl'):

    with open(out_file, 'w') as f:
        for low in range(0, len(data), n):
            high = low + n
            text_list = data['text'].values.tolist()[low:high]
            label_list = data['label'].values.tolist()[low:high]
            
            # prompts
            system_prompt = """Du får udleveret 20 tekststykker. Du skal vurdere, om hver tekst er skrevet af et menneske eller en AI. Du skal svare 0, hvis du tror, ​​det er skrevet af et menneske, og 1, hvis du tror, ​​det er skrevet af en AI. Kun ét ciffer per text er tilladt. Du vil få input som er afmærket med et tal og 3 dollartegn ($$$) i starten og slutningen. Der skal til mellem hvert svar være et linjeskift (\n).
                    
                    Eksempel:
                    1. $$$Tekst 1 er en lang text ....
                    den dækker flere linjer
                    Og har alverdens tegn i sig
                    $$$
                    2. $$$Tekst 2$$$

                    3. $$$Tekst 3$$$
                    ...

                    Og du skal svare med:
                    1. 0
                    2. 1
                    3. 0
                    ...
                    """

            text_list_formatted = [str(i+1) + '. $$$' + text + '$$$' for i, text in enumerate(text_list)]
            text = '\n\n'.join(text_list_formatted)
            print(text)

            labels = '\n'.join([str(i+1) + '. ' + str(x) for i, x in enumerate(label_list)])
            print(labels)

            
            json_line = json.dumps({"messages": [
                {"role": "system", "content": system_prompt}, 
                {"role": "user", "content": text},
                {"role": "assistant", "content": str(labels)}
            ]})
            f.write(json_line + '\n')

save_to_json(data, n=20, out_file='data_train.jsonl')
save_to_json(data_val, n=20, out_file='data_val.jsonl')

1. $$$du er her: Details > Om os > Erhvervsaftaler
Erhvervskunder og myndigheder, der regelmæssigt anvender vores oversættelsestjeneste, kan drage fordel af en aftale, der tilbyder mange ekstra ydelser! Det er blot en måde, hvorpå vi kan arbejde effektivt og succesfuldt sammen med dig.
Hvad omfatter en erhvervsaftale?
Erhvervskunder med en erhvervsaftale kan uploade tekster, der skal oversættes, direkte ind i vores håndteringssystem til oversættelser og hente dem samme sted$$$

2. $$$Ja, det forlyder, at der ikke længere er en udvælgelsesproces for deltagelse i Transalp Challenge. Tidligere var der visse kriterier såsom først-til-mølle-princippet og fortrinsret til visse kategorier, men det ser ud til at være ændret, og nu er det en simpel lodtrækning, der afgør, hvem der får en plads. Så det handler mere om held og tilfældigheder end om at træne sig til udtagelse. Det kan være lidt ærgerligt, hvis man har arbejdet hårdt for at komme i form og gerne vil deltage i løbet.$$$

3. $$$Famil

In [130]:
# Upload to OpenAI
from openai import OpenAI
client = OpenAI()

client.files.create(
    file=open("data_train.jsonl", "rb"),
    purpose="fine-tune"
)


FileObject(id='file-7KOtSkouG66AoWgKNsbu65wZ', bytes=1115518, created_at=1700181699, filename='data_train.jsonl', object='file', purpose='fine-tune', status='processed', status_details=None)

In [131]:
client.files.create(
    file=open("data_val.jsonl", "rb"),
    purpose="fine-tune"
)

FileObject(id='file-pHo0UyAA667iig4HgyNLVACG', bytes=223985, created_at=1700181701, filename='data_val.jsonl', object='file', purpose='fine-tune', status='processed', status_details=None)

In [132]:
from openai import OpenAI
client = OpenAI()

client.fine_tuning.jobs.create(
  training_file="file-7KOtSkouG66AoWgKNsbu65wZ",
  validation_file="file-pHo0UyAA667iig4HgyNLVACG",
    model="gpt-3.5-turbo-1106",
)

FineTuningJob(id='ftjob-dC5JCJHzYzrnx7AD5hArXjgP', created_at=1700181717, error=None, fine_tuned_model=None, finished_at=None, hyperparameters=Hyperparameters(n_epochs='auto', batch_size='auto', learning_rate_multiplier='auto'), model='gpt-3.5-turbo-1106', object='fine_tuning.job', organization_id='org-87Ypq9qoCRxP9EESHWe2tRdz', result_files=[], status='validating_files', trained_tokens=None, training_file='file-7KOtSkouG66AoWgKNsbu65wZ', validation_file='file-pHo0UyAA667iig4HgyNLVACG')

## Apply to val set

In [134]:
data_val

,text,label
918,"Jeg er ikke Kinaspecialist, men mit bud er, at...",0
919,"Nej, lige nu arbejder diplomatiet ikke nær så ...",0
920,Jeg har altid været åben for at indgå i en reg...,1
921,En typisk dag for en ph.d.-studerende er ret v...,1
922,"Det er svært at sige, da vi ikke er direkte in...",1
...,...,...
1128,"Ja, det er blevet rapporteret, at Rusland står...",1
1129,"Ja, der er tale om en domsmandssag, hvor der e...",1
1130,Vi er sociale liberale - det er ikke at man er...,1
1131,"I klubberne er der også fokus på præstation, o...",1


In [145]:
text_batches_val = []
batch_size = 20
for i in range(0, len(data_val), 20):
    text_batches_val.append(data_val['text'].values.tolist()[i:i+batch_size])

print(len(text_batches_val))
text_batches_val = text_batches_val*10
print(len(text_batches_val))

11
110


In [146]:
# use fine-tuned model
model_finetuned= "ft:gpt-3.5-turbo-1106:personal::8LhXcRJg"

pred = predict_parallel(text_batches_val)
pred_concat = np.concatenate(pred)
pred_concat

Rate limit reached, sleeping for 30 milliseconds
Rate limit reached, sleeping for 30 milliseconds
Rate limit reached, sleeping for 30 milliseconds
Rate limit reached, sleeping for 30 milliseconds
Rate limit reached, sleeping for 30 milliseconds
Rate limit reached, sleeping for 30 milliseconds
Rate limit reached, sleeping for 30 milliseconds
Rate limit reached, sleeping for 30 milliseconds
Rate limit reached, sleeping for 30 milliseconds
Rate limit reached, sleeping for 30 milliseconds
Rate limit reached, sleeping for 30 milliseconds
Rate limit reached, sleeping for 30 milliseconds
Rate limit reached, sleeping for 30 milliseconds
Rate limit reached, sleeping for 30 milliseconds
Rate limit reached, sleeping for 30 milliseconds
Rate limit reached, sleeping for 30 milliseconds
Rate limit reached, sleeping for 30 milliseconds
Rate limit reached, sleeping for 30 milliseconds
Rate limit reached, sleeping for 30 milliseconds
Rate limit reached, sleeping for 30 milliseconds
Rate limit reached, 

array([0, 1, 1, ..., 0, 1, 1])

In [141]:
data_val['pred'] = pred_concat

# confusion matrix
from sklearn.metrics import confusion_matrix
print(confusion_matrix(data_val['label'].astype(int).values.tolist(), data_val['pred'].values.tolist()))

# accuracy
from sklearn.metrics import accuracy_score
print(accuracy_score(data_val['label'].astype(int).values.tolist(), data_val['pred'].values.tolist()))


[[91 21]
 [ 4 99]]
0.8837209302325582
